In [8]:
import cdsapi

dataset = "derived-era5-single-levels-daily-statistics"
request = {
    "product_type": "reanalysis",
    "variable": ["2m_temperature"],
    "year": "1980",
    "month": ["01"],
    "day": ["01", "02"],
    "daily_statistic": "daily_maximum",
    "time_zone": "utc-07:00",
    "frequency": "1_hourly",
    "area": [62, -129, 47, -105]
}

client = cdsapi.Client()
client.retrieve(dataset, request, "era5_test_data.grib")


2025-11-23 11:39:47,274 INFO Request ID is 347af7ef-4799-4e71-850c-24912e5c7f21
2025-11-23 11:39:47,469 INFO status has been updated to accepted
2025-11-23 11:40:01,642 INFO status has been updated to running
2025-11-23 11:40:09,438 INFO status has been updated to accepted
2025-11-23 11:40:21,032 INFO status has been updated to successful


96f587dacf1c0cf6c6533d6af2cb0042.nc:   0%|          | 0.00/46.3k [00:00<?, ?B/s]

'era5_test_data.grib'